In [ ]:
# Marp CLIをNode.js経由でグローバルにインストールします
!npm install -g @marp-team/marp-cli@2.4.0

# PDF生成時に日本語が正しく表示されるように、Notoフォントをインストールします
!apt-get install -y fonts-noto-cjk

print("\n✅ 環境設定が完了しました。次のセルに進んでください。")

In [ ]:
import subprocess
import os
from pathlib import Path
import shutil
import base64
import mimetypes
from IPython.display import display, HTML, clear_output
from google.colab import files
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout

# --- 定数とセットアップ ---
OUTPUT_DIR = Path("output_colab")
OUTPUT_DIR.mkdir(exist_ok=True)

# Marp CLIの実行可能ファイルパスを取得
# Colabではグローバルにインストールされるため、shutil.whichで簡単に見つかります
MARP_PATH = shutil.which("marp")
if not MARP_PATH:
    raise RuntimeError("Marp CLIが見つかりません。セル1が正しく実行されたか確認してください。")

COLOR_THEMES = {
    "ブルー/イエロー (デフォルト)": {
        "header_bg": "linear-gradient(135deg, #003f7f 0%,rgb(0, 120, 212) 100%)",
        "info_bg": "linear-gradient(135deg, #ffd700 0%,hsl(51, 98.50%, 73.90%) 100%)",
        "info_border": "#f0c814",
        "info_shadow": "rgba(255, 215, 0, 0.3)",
        "main_text": "#003f7f",
    },
    "グリーン/オレンジ": {
        "header_bg": "linear-gradient(135deg, #005a32 0%, #1e8449 100%)",
        "info_bg": "linear-gradient(135deg, #f39c12 0%, #e67e22 100%)",
        "info_border": "#d35400",
        "info_shadow": "rgba(243, 156, 18, 0.3)",
        "main_text": "#005a32",
    },
    "パープル/ミント": {
        "header_bg": "linear-gradient(135deg, #4a148c 0%, #8e44ad 100%)",
        "info_bg": "linear-gradient(135deg, #a7ffeb 0%, #64ffda 100%)",
        "info_border": "#1de9b6",
        "info_shadow": "rgba(100, 255, 218, 0.3)",
        "main_text": "#4a148c",
    },
    "モノクローム": {
        "header_bg": "linear-gradient(135deg, #2c3e50 0%, #34495e 100%)",
        "info_bg": "linear-gradient(135deg, #ecf0f1 0%, #bdc3c7 100%)",
        "info_border": "#95a5a6",
        "info_shadow": "rgba(189, 195, 199, 0.3)",
        "main_text": "#2c3e50",
    }
}

# --- ヘルパー関数 ---
def get_dynamic_font_size(text, base_size=1.5, min_size=0.8, shrink_factor=12):
    if len(text) > shrink_factor:
        reduction = (len(text) - shrink_factor) * 0.1
        return max(min_size, base_size - reduction)
    return base_size

# --- Markdown生成関数 (元のコードから変更なし) ---
def generate_markdown(
    colloquium_name, title, photo_path, speaker_name, affiliation,
    date_time, location, abstract, colors, abstract_font_size, abstract_height,
    title_font_size, speaker_font_size
):
    style_css = f"""
<style>
  @import url('https://fonts.googleapis.com/css2?family=Noto+Sans+JP:wght@400;500;700&display=swap');
  section {{
    display: flex; flex-direction: column; padding: 0;
    font-family: 'Noto Sans JP', sans-serif;
    background: linear-gradient(135deg, #f1f3f6 0%, #e8eef4 100%);
    overflow: hidden;
  }}
  .header-container {{
    background: {colors['header_bg']};
    padding: 50px 60px 30px 60px; margin-bottom: 0px;
    transform: skewY(-3deg); position: relative; z-index: 1; margin-top: -30px;
  }}
  .colloquium-name {{
    position: absolute; top: 35px; left: 970px; font-size: 1em;
    font-weight: 500; color: rgba(255, 255, 255, 0.8); transform: skewY(3deg);
  }}
  .title {{
    font-size: {title_font_size}em; font-weight: 900; color: #ffffff; text-align: center;
    line-height: 1.3; text-shadow: 0 2px 5px rgba(0, 0, 0, 0.25);
    transform: skewY(3deg); margin-bottom: 0; white-space: nowrap;
  }}
  .main-content {{
    display: flex; flex-direction: row; flex: 1; gap: 50px; padding: 0 40px;
  }}
  .left-panel {{
    flex: 0 0 280px; display: flex; flex-direction: column; align-items: center;
  }}
  .speaker-photo {{
    width: 250px; height: 250px; border-radius: 50%; object-fit: cover;
    object-position: center;
    box-shadow: 0 10px 30px rgba(0, 63, 127, 0.25);
    border: 5px solid #ffffff; margin-bottom: 25px; position: relative; z-index: 2;
  }}
  .speaker-info {{
    text-align: center; background-color: white; padding: 20px;
    border-radius: 12px; box-shadow: 0 4px 12px rgba(0, 0, 0, 0.1); width: 100%;
  }}
  .speaker-name {{
    font-size: {speaker_font_size}em; font-weight: 700; color: {colors['main_text']}; margin-bottom: 8px;
    white-space: nowrap;
  }}
  .affiliation {{
    font-size: 1.0em; color: #666; line-height: 1.4; font-weight: 400;
  }}
  .right-panel {{ flex: 1; display: flex; flex-direction: column; gap: 30px; }}
  .info-section {{
    background: {colors['info_bg']};
    padding: 25px; border-radius: 12px;
    box-shadow: 0 4px 12px {colors['info_shadow']};
    border: 2px solid {colors['info_border']};
  }}
  .event-info {{
    display: grid; grid-template-columns: auto 1fr; gap: 15px 25px; font-size: 0.9em;
  }}
  .info-label {{ font-weight: 700; color: {colors['main_text']}; }}
  .info-value {{ color: {colors['main_text']}; font-weight: 500; }}
  .abstract {{
    background-color: white; padding: 25px; border-radius: 12px;
    box-shadow: 0 4px 12px rgba(1, 36, 72, 0.15);
    border-left: 4px solid {colors['main_text']};
    height: {abstract_height}px;
    overflow-y: auto;
  }}
  .abstract-title {{
    font-size: 0.8em; color: {colors['main_text']}; margin-bottom: 18px;
    font-weight: 700; border-bottom: 2px solid {colors['main_text']}; padding-bottom: 8px;
  }}
  .abstract p {{
    font-size: {abstract_font_size}em; line-height: 1.6; color: #333; text-align: justify;
    margin-bottom: 12px; font-weight: 400;
  }}
</style>
"""
    affiliation_html = affiliation.replace('\n', '<br>')
    abstract_html = f"<p>{' '.join(abstract.strip().splitlines())}</p>"
    content_html = f"""
<div class="header-container">
  <div class="colloquium-name">{colloquium_name}</div>
  <div class="title">{title}</div>
</div>
<div class="main-content">
  <div class="left-panel">
    <img src="{photo_path}" alt="講演者写真" class="speaker-photo">
    <div class="speaker-info">
      <div class="speaker-name">{speaker_name}</div>
      <div class="affiliation">{affiliation_html}</div>
    </div>
  </div>
  <div class="right-panel">
    <div class="info-section">
      <div class="event-info">
        <div class="info-label">日時：</div> <div class="info-value">{date_time}</div>
        <div class="info-label">場所：</div> <div class="info-value">{location}</div>
      </div>
    </div>
    <div class="abstract">
      <div class="abstract-title">講演概要</div>
      {abstract_html}
    </div>
  </div>
</div>
"""
    return f"""---
marp: true
theme: default
paginate: false
size: 16:9
---
{style_css}
{content_html}
"""

print("✅ ライブラリのインポートと関数の準備が完了しました。次のセルに進んでください。")
Use code with caution.
Python
セル 3: インタラクティブUIの表示とポスター生成
このセルを実行すると、情報の入力欄とプレビューが表示されます。値を変更すると、プレビューが自動的に更新されます。写真のアップロードもここで行います。
Generated python
#
# ----- セル 3: インタラクティブUIの表示とポスター生成 -----
#

# --- 1. UIウィジェットの作成 ---
style = {'description_width': '150px'}
layout = Layout(width='80%')

# ポスター情報
w_colloquium_name = widgets.Text(description="コロキウム名:", value="物理学教室コロキウム", style=style, layout=layout)
w_title = widgets.Text(description="講演タイトル:", value="高分子ゲルの精密な物理学", style=style, layout=layout)
w_speaker_name = widgets.Text(description="講演者名:", value="酒井 崇匡 氏", style=style, layout=layout)
w_affiliation = widgets.Textarea(description="所属:", value="東京大学大学院\n理学系研究科", style=style, layout=layout)
w_date_time = widgets.Text(description="日時:", value="2025年6月20日（金）17:00-18:30", style=style, layout=layout)
w_location = widgets.Text(description="場所:", value="小柴ホール", style=style, layout=layout)
w_uploaded_photo = widgets.FileUpload(description="講演者の写真:", accept='.jpg, .jpeg, .png', multiple=False, style=style)
w_abstract = widgets.Textarea(description="講演概要:", value="ハイドロゲル（以下ゲルと略）は、多量の水で膨潤した高分子ネットワークであり...（以下略）", style=style, layout=Layout(width='80%', height='150px'))

# デザイン設定
w_selected_theme_name = widgets.Dropdown(description="カラーテーマ:", options=COLOR_THEMES.keys(), value="ブルー/イエロー (デフォルト)", style=style, layout=layout)
w_title_font_size = widgets.FloatSlider(description="タイトルのフォントサイズ:", min=1.0, max=4.0, step=0.1, value=2.8, style=style, layout=layout)
w_abstract_font_size = widgets.FloatSlider(description="概要のフォントサイズ:", min=0.5, max=1.0, step=0.05, value=0.65, style=style, layout=layout)
w_abstract_height = widgets.IntSlider(description="概要ボックスの高さ (px):", min=100, max=500, step=10, value=250, style=style, layout=layout)

# プレビューとダウンロード用の出力エリア
preview_output = widgets.Output()
download_output = widgets.Output()

# --- 2. プレビューとPDFを生成する関数 ---
def generate_poster_on_change(change):
    # 出力エリアをクリア
    preview_output.clear_output(wait=True)
    download_output.clear_output(wait=True)
    
    # 画像の処理
    if w_uploaded_photo.value:
        # アップロードされたファイル情報を取得
        uploaded_file = w_uploaded_photo.value[0]
        image_bytes = uploaded_file['content']
        mime_type = mimetypes.guess_type(uploaded_file['name'])[0]
        photo_display_path = f"data:{mime_type};base64,{base64.b64encode(image_bytes).decode()}"
        photo_uploaded = True
    else:
        # プレースホルダー画像
        placeholder_svg = """<svg width="250" height="250" viewBox="0 0 250 250" xmlns="http://www.w3.org/2000/svg"><rect width="100%" height="100%" fill="#e9ecef"/><text x="50%" y="50%" dominant-baseline="middle" text-anchor="middle" font-family="sans-serif" font-size="20" fill="#6c757d">写真なし</text></svg>"""
        photo_display_path = f"data:image/svg+xml;base64,{base64.b64encode(placeholder_svg.encode()).decode()}"
        photo_uploaded = False

    # フォントサイズの動的調整
    speaker_font_size = get_dynamic_font_size(w_speaker_name.value)

    # Markdownコンテンツの生成
    markdown_content = generate_markdown(
        w_colloquium_name.value, w_title.value, photo_display_path, w_speaker_name.value,
        w_affiliation.value, w_date_time.value, w_location.value, w_abstract.value,
        COLOR_THEMES[w_selected_theme_name.value], w_abstract_font_size.value,
        w_abstract_height.value, w_title_font_size.value, speaker_font_size
    )

    md_path = OUTPUT_DIR / "poster.md"
    html_path = OUTPUT_DIR / "preview.html"
    pdf_path = OUTPUT_DIR / "poster.pdf"
    md_path.write_text(markdown_content, encoding="utf-8")

    # --- プレビュー (HTML) 生成 ---
    with preview_output:
        try:
            cmd_html = [MARP_PATH, str(md_path), "-o", str(html_path), "--html", "--allow-local-files"]
            subprocess.run(cmd_html, check=True, capture_output=True, text=True, encoding='utf-8')
            with open(html_path, "r", encoding="utf-8") as f:
                html_content = f.read()
            
            # Colabの表示領域に合わせてHTMLのサイズを調整
            scaled_html = f'<div style="transform: scale(0.85); transform-origin: top left; width: 117%; height: 550px;">{html_content}</div>'
            display(HTML(scaled_html))
        except Exception as e:
            print("プレビューの生成に失敗しました。")
            if hasattr(e, 'stderr'):
                print("--- エラー詳細 ---")
                print(e.stderr)

    # --- PDF生成とダウンロードボタン ---
    with download_output:
        if photo_uploaded:
            try:
                # PDF生成
                cmd_pdf = [MARP_PATH, str(md_path), "-o", str(pdf_path), "--pdf", "--allow-local-files"]
                subprocess.run(cmd_pdf, check=True, capture_output=True, text=True, encoding='utf-8')
                
                # ダウンロードボタンの作成
                print(f"✅ PDFが生成されました: {pdf_path}")
                print("以下のボタンをクリックしてダウンロードしてください。")
                download_button = widgets.Button(description="ポスターPDFをダウンロード", button_style='primary')

                def on_download_button_clicked(b):
                    files.download(str(pdf_path))
                
                download_button.on_click(on_download_button_clicked)
                display(download_button)

            except Exception as e:
                print("PDFの生成に失敗しました。")
                if hasattr(e, 'stderr'):
                    print("--- エラー詳細 ---")
                    print(e.stderr)
        else:
            display(HTML("<p style='color: orange;'><b>警告:</b> PDFを生成・ダウンロードするには、講演者の写真をアップロードしてください。</p>"))


# --- 3. UIのレイアウトと表示 ---

# 各ウィジェットの変更を監視し、関数を呼び出す
widgets_to_observe = [
    w_colloquium_name, w_title, w_speaker_name, w_affiliation, w_date_time,
    w_location, w_uploaded_photo, w_abstract, w_selected_theme_name,
    w_title_font_size, w_abstract_font_size, w_abstract_height
]
for w in widgets_to_observe:
    w.observe(generate_poster_on_change, names='value')

# UIのレイアウト
inputs_box = VBox([
    widgets.HTML("<h3>ポスター情報入力</h3>"),
    w_colloquium_name, w_title, w_speaker_name, w_affiliation, w_date_time, w_location,
    w_uploaded_photo, w_abstract,
    widgets.HTML("<hr><h3>デザイン設定</h3>"),
    w_selected_theme_name, w_title_font_size, w_abstract_font_size, w_abstract_height
])

main_content_box = VBox([
    widgets.HTML("<h2>リアルタイムプレビュー</h2>"),
    preview_output,
    widgets.HTML("<hr><h2>PDF生成・ダウンロード</h2>"),
    download_output
])

# サイドバー風のレイアウト
ui = HBox([
    inputs_box,
    main_content_box
], layout=Layout(width='100%', justify_content='space-between'))

# 初期表示
display(ui)
generate_poster_on_change(None) # 最初のプレビューを生成

In [ ]:
# --- 1. UIウィジェットの作成 ---
style = {'description_width': '150px'}
layout = Layout(width='80%')

# ポスター情報
w_colloquium_name = widgets.Text(description="コロキウム名:", value="物理学教室コロキウム", style=style, layout=layout)
w_title = widgets.Text(description="講演タイトル:", value="高分子ゲルの精密な物理学", style=style, layout=layout)
w_speaker_name = widgets.Text(description="講演者名:", value="酒井 崇匡 氏", style=style, layout=layout)
w_affiliation = widgets.Textarea(description="所属:", value="東京大学大学院\n理学系研究科", style=style, layout=layout)
w_date_time = widgets.Text(description="日時:", value="2025年6月20日（金）17:00-18:30", style=style, layout=layout)
w_location = widgets.Text(description="場所:", value="小柴ホール", style=style, layout=layout)
w_uploaded_photo = widgets.FileUpload(description="講演者の写真:", accept='.jpg, .jpeg, .png', multiple=False, style=style)
w_abstract = widgets.Textarea(description="講演概要:", value="ハイドロゲル（以下ゲルと略）は、多量の水で膨潤した高分子ネットワークであり...（以下略）", style=style, layout=Layout(width='80%', height='150px'))

# デザイン設定
w_selected_theme_name = widgets.Dropdown(description="カラーテーマ:", options=COLOR_THEMES.keys(), value="ブルー/イエロー (デフォルト)", style=style, layout=layout)
w_title_font_size = widgets.FloatSlider(description="タイトルのフォントサイズ:", min=1.0, max=4.0, step=0.1, value=2.8, style=style, layout=layout)
w_abstract_font_size = widgets.FloatSlider(description="概要のフォントサイズ:", min=0.5, max=1.0, step=0.05, value=0.65, style=style, layout=layout)
w_abstract_height = widgets.IntSlider(description="概要ボックスの高さ (px):", min=100, max=500, step=10, value=250, style=style, layout=layout)

# プレビューとダウンロード用の出力エリア
preview_output = widgets.Output()
download_output = widgets.Output()

# --- 2. プレビューとPDFを生成する関数 ---
def generate_poster_on_change(change):
    # 出力エリアをクリア
    preview_output.clear_output(wait=True)
    download_output.clear_output(wait=True)
    
    # 画像の処理
    if w_uploaded_photo.value:
        # アップロードされたファイル情報を取得
        uploaded_file = w_uploaded_photo.value[0]
        image_bytes = uploaded_file['content']
        mime_type = mimetypes.guess_type(uploaded_file['name'])[0]
        photo_display_path = f"data:{mime_type};base64,{base64.b64encode(image_bytes).decode()}"
        photo_uploaded = True
    else:
        # プレースホルダー画像
        placeholder_svg = """<svg width="250" height="250" viewBox="0 0 250 250" xmlns="http://www.w3.org/2000/svg"><rect width="100%" height="100%" fill="#e9ecef"/><text x="50%" y="50%" dominant-baseline="middle" text-anchor="middle" font-family="sans-serif" font-size="20" fill="#6c757d">写真なし</text></svg>"""
        photo_display_path = f"data:image/svg+xml;base64,{base64.b64encode(placeholder_svg.encode()).decode()}"
        photo_uploaded = False

    # フォントサイズの動的調整
    speaker_font_size = get_dynamic_font_size(w_speaker_name.value)

    # Markdownコンテンツの生成
    markdown_content = generate_markdown(
        w_colloquium_name.value, w_title.value, photo_display_path, w_speaker_name.value,
        w_affiliation.value, w_date_time.value, w_location.value, w_abstract.value,
        COLOR_THEMES[w_selected_theme_name.value], w_abstract_font_size.value,
        w_abstract_height.value, w_title_font_size.value, speaker_font_size
    )

    md_path = OUTPUT_DIR / "poster.md"
    html_path = OUTPUT_DIR / "preview.html"
    pdf_path = OUTPUT_DIR / "poster.pdf"
    md_path.write_text(markdown_content, encoding="utf-8")

    # --- プレビュー (HTML) 生成 ---
    with preview_output:
        try:
            cmd_html = [MARP_PATH, str(md_path), "-o", str(html_path), "--html", "--allow-local-files"]
            subprocess.run(cmd_html, check=True, capture_output=True, text=True, encoding='utf-8')
            with open(html_path, "r", encoding="utf-8") as f:
                html_content = f.read()
            
            # Colabの表示領域に合わせてHTMLのサイズを調整
            scaled_html = f'<div style="transform: scale(0.85); transform-origin: top left; width: 117%; height: 550px;">{html_content}</div>'
            display(HTML(scaled_html))
        except Exception as e:
            print("プレビューの生成に失敗しました。")
            if hasattr(e, 'stderr'):
                print("--- エラー詳細 ---")
                print(e.stderr)

    # --- PDF生成とダウンロードボタン ---
    with download_output:
        if photo_uploaded:
            try:
                # PDF生成
                cmd_pdf = [MARP_PATH, str(md_path), "-o", str(pdf_path), "--pdf", "--allow-local-files"]
                subprocess.run(cmd_pdf, check=True, capture_output=True, text=True, encoding='utf-8')
                
                # ダウンロードボタンの作成
                print(f"✅ PDFが生成されました: {pdf_path}")
                print("以下のボタンをクリックしてダウンロードしてください。")
                download_button = widgets.Button(description="ポスターPDFをダウンロード", button_style='primary')

                def on_download_button_clicked(b):
                    files.download(str(pdf_path))
                
                download_button.on_click(on_download_button_clicked)
                display(download_button)

            except Exception as e:
                print("PDFの生成に失敗しました。")
                if hasattr(e, 'stderr'):
                    print("--- エラー詳細 ---")
                    print(e.stderr)
        else:
            display(HTML("<p style='color: orange;'><b>警告:</b> PDFを生成・ダウンロードするには、講演者の写真をアップロードしてください。</p>"))


# --- 3. UIのレイアウトと表示 ---

# 各ウィジェットの変更を監視し、関数を呼び出す
widgets_to_observe = [
    w_colloquium_name, w_title, w_speaker_name, w_affiliation, w_date_time,
    w_location, w_uploaded_photo, w_abstract, w_selected_theme_name,
    w_title_font_size, w_abstract_font_size, w_abstract_height
]
for w in widgets_to_observe:
    w.observe(generate_poster_on_change, names='value')

# UIのレイアウト
inputs_box = VBox([
    widgets.HTML("<h3>ポスター情報入力</h3>"),
    w_colloquium_name, w_title, w_speaker_name, w_affiliation, w_date_time, w_location,
    w_uploaded_photo, w_abstract,
    widgets.HTML("<hr><h3>デザイン設定</h3>"),
    w_selected_theme_name, w_title_font_size, w_abstract_font_size, w_abstract_height
])

main_content_box = VBox([
    widgets.HTML("<h2>リアルタイムプレビュー</h2>"),
    preview_output,
    widgets.HTML("<hr><h2>PDF生成・ダウンロード</h2>"),
    download_output
])

# サイドバー風のレイアウト
ui = HBox([
    inputs_box,
    main_content_box
], layout=Layout(width='100%', justify_content='space-between'))

# 初期表示
display(ui)
generate_poster_on_change(None) # 最初のプレビューを生成